In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

import seaborn as sns
from matplotlib import pyplot as plt
import os
print(os.getcwd())
#os.chdir("..\..\lib")
print(os.getcwd())
from datetime import datetime

import ingest, predict, models_base
%load_ext autoreload
%autoreload 1

## FLAGS ##
# RAS model output or USGS
USGS_FLAG = True

### HENRY RAS ###
csv = r"..\data\Henry_WSS_2017_2023.csv"
columns = {'Q': 'Discharge', 'WSS': 'Slope'}

date = datetime.now().strftime("%B %d %Y - %H:%M")


target = "Discharge"
data_name = f"Henry_RAS_2017_2023 {date}"


'''
### HENRY ###
csv = r"..\\data\\henry_csv_17-23.csv"
#renames = {'00065': 'Gage Height', '00060': 'Discharge', '72254': 'Velocity'}
columns = {'65': 'Gage Height', '60': 'Discharge', '72254': 'Velocity'}

target = "Discharge"
data_name = "Henry_2017_2020"
'''
'''

csv = "..\\data\\USGS_WS_2017_2023.csv"
columns = {
           "Peoria_WL": "Peoria_WL",
           "Henry_WL": "Henry_WL",
#          "Flow": "Flow",       --- Target
#          "Vel": "Vel",         --- Disregard
           "Slope": "Slope"
           }

target = "Flow"
data_name = "USGS_WS_2017_2023"
'''



c:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib\testing_scratch
c:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib


'\n\ncsv = "..\\data\\USGS_WS_2017_2023.csv"\ncolumns = {\n           "Peoria_WL": "Peoria_WL",\n           "Henry_WL": "Henry_WL",\n#          "Flow": "Flow",       --- Target\n#          "Vel": "Vel",         --- Disregard\n           "Slope": "Slope"\n           }\n\ntarget = "Flow"\ndata_name = "USGS_WS_2017_2023"\n'

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
%aimport models_cuda


In [6]:

train_scaled, test_scaled, train_dates, test_dates, all_dates, scaler = ingest.ingest(csv, target, renames= columns, USGS_FLAG=USGS_FLAG, train_test_ratio= 0.8)
trainX, trainY = ingest.reshape(train_scaled)#, timestep_type= "hr")
testX, testY = ingest.reshape(test_scaled)#, timestep_type= "hr")

model_names = ['Basic_LSTM', "GRU", 'Bidirectional_LSTM','Stacked_LSTM']


c:\Users\Mikey\Documents\Github\Hysterisis-ML-Modeling\lib\ingest.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_range, test_range = [all_dates[0], all_dates[-1]], [all_dates[0], all_dates[-1]]


In [11]:
all_dates["1/1/2017 0:00":"12/31/2021 23:45"]

datetime
1/1/2017 0:00          1/1/2017 0:00
1/1/2017 0:15          1/1/2017 0:15
1/1/2017 0:30          1/1/2017 0:30
1/1/2017 0:45          1/1/2017 0:45
1/1/2017 1:00          1/1/2017 1:00
                          ...       
12/31/2021 22:45    12/31/2021 22:45
12/31/2021 23:00    12/31/2021 23:00
12/31/2021 23:15    12/31/2021 23:15
12/31/2021 23:30    12/31/2021 23:30
12/31/2021 23:45    12/31/2021 23:45
Name: datetime, Length: 175296, dtype: object

In [39]:


for model_name in model_names:
    model = models_base.prebuilt_models(model_name, trainX, trainY, epochs= 10, batch_size=16, load_models=False, data= data_name)
    validation_loss = models_base.evaluate_model(model, testX, testY)
    models_base.plot_model(model_name, validation_loss, data_name)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 125)               64000     
                                                                 
 dense_1 (Dense)             (None, 1)                 126       
                                                                 
Total params: 64,126
Trainable params: 64,126
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  350/10667 [..............................] - ETA: 2:51 - loss: 0.0338

KeyboardInterrupt: 

In [31]:

'''
for model_name in model_names:
    model = models_cuda.prebuilt_models(model_name, trainX, trainY, epochs= 10, batch_size=16, load_models=False, data= data_name)
    validation_loss = models_cuda.evaluate_model(model, testX, testY)
    models_cuda.plot_model(model_name, validation_loss, data_name)
'''


NameError: name 'Sequential' is not defined

In [32]:
test_dates

datetime
6/2/2022 10:00      6/2/2022 10:00
6/2/2022 10:15      6/2/2022 10:15
6/2/2022 10:30      6/2/2022 10:30
6/2/2022 10:45      6/2/2022 10:45
6/2/2022 11:00      6/2/2022 11:00
                        ...       
10/9/2023 23:00    10/9/2023 23:00
10/9/2023 23:15    10/9/2023 23:15
10/9/2023 23:30    10/9/2023 23:30
10/9/2023 23:45    10/9/2023 23:45
10/10/2023 0:00    10/10/2023 0:00
Name: datetime, Length: 47481, dtype: object

November 07 2023 - 09:01


In [35]:
%aimport predict

In [36]:
tstart = '3/18/2022 0:00'
tend = '4/7/2022 0:00'
event_range = [tstart, tend]

for model_name in model_names:
    #predicts = predict.predict(model_name, testX, data_name)
    predict.plot_predicts(model_name, predicts, testY, test_dates, data_name, event_range= event_range, event_plotstep= "Day")




ValueError: Event, ['3/18/2022 0:00', '4/7/2022 0:00'], not in test set date range. Please choose another range.